In [ ]:
#Importing libraries

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import re
import numpy as np
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium import webdriver
import codecs
import os
from selenium.webdriver.support.ui import WebDriverWait
from pymongo import MongoClient
from selenium.webdriver.support import expected_conditions
import http.client, urllib.parse
from pymongo import MongoClient
from urllib.parse import quote_plus
from dataprep.datasets import load_dataset
from dataprep.eda import create_report
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from fancyimpute import KNN
from statistics import mode
from collections import Counter
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBClassifier, plot_tree
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor

In [4]:
from pymongo import MongoClient
#Connecting to mongoDB
mo_c = MongoClient()
client = MongoClient('localhost', 27017)
#access database
db = client["craigslist_cars"]
collection_sf=db["bay_area"]
collection_sd=db['sd']
collection_la=db['los_angeles']

bay_area_df = pd.DataFrame(list(collection_sf.find({})))
san_diego_df = pd.DataFrame(list(collection_sd.find({})))
los_angeles_df = pd.DataFrame(list(collection_la.find({})))

In [5]:
#Concat 3 dfs to create a full df
full_df = pd.DataFrame()
full_df = pd.concat([bay_area_df, san_diego_df, los_angeles_df])

full_df.drop('posted_stats', axis=1, inplace=True)

In [13]:
full_df.head()

,_id,url,full_car_brand,year,brand,fuel,odo,cond,paint,cyl,title,trans,drive,body_type,img_url,price,size
0,641404f459aa7868e84596fd,https://sfbay.craigslist.org/sfc/cto/d/san-fra...,Honda Element 2008 EX,2008.0,honda,gas,156900.0,excellent,grey,4.0,clean,manual,fwd,SUV,https://images.craigslist.org/00707_6t1Lg18wmo...,4500.0,NaN
1,641404f459aa7868e84596fe,https://sfbay.craigslist.org/eby/cto/d/martine...,2009 kia spectra,2009.0,kia,gas,152000.0,good,red,4.0,clean,automatic,fwd,sedan,https://images.craigslist.org/00b0b_jKLhf1Pv6U...,3850.0,mid-size
2,641404f459aa7868e84596ff,https://sfbay.craigslist.org/eby/cto/d/oakland...,2012 Subaru Impreza,2012.0,subaru,gas,133000.0,good,white,NaN,clean,automatic,4wd,hatchback,https://images.craigslist.org/00d0d_c8TotKANf7...,9800.0,NaN
3,641404f459aa7868e8459700,https://sfbay.craigslist.org/sby/cto/d/san-jos...,2016 smart car prime low mile,2016.0,NaN,gas,33000.0,like new,blue,3.0,clean,automatic,rwd,other,https://images.craigslist.org/00000_g6c68odvnk...,11000.0,compact
4,641404f459aa7868e8459701,https://sfbay.craigslist.org/eby/cto/d/hayward...,2015 FORD TRANSIT CONNECT XLT CARGO VAN,2015.0,ford,gas,150000.0,NaN,NaN,NaN,clean,automatic,NaN,NaN,https://images.craigslist.org/00101_jaSFzUnrP7...,14200.0,NaN


In [18]:
full_df.to_csv('price_prediction_df.csv')
#optional:read in file from csv:
#full_df=pd.read_csv('price_prediction_df.csv')

In [ ]:
##CLEANING
full_df = full_df.dropna(subset=['full_car_brand','fuel','odo','title','trans','img_url']) 
df1 = full_df.loc[:, ['year', 'odo', 'cyl', 'price']]
df2 = full_df.drop(['year', 'odo', 'cyl', 'price'], axis=1)

##NUMERIC KNN IMPUTING
df1 = df1.apply(pd.to_numeric, errors='coerce')

#Need to hyperparametrize n here
# apply KNNImputer to the dataframe
imputer = KNNImputer(n_neighbors=2)
df1_imputed = pd.DataFrame(imputer.fit_transform(df1), columns=df1.columns)

# print the resulting dataframe
print(df1_imputed)

df = df2.join(df1_imputed)
df = df.drop(['size','drive','paint'],axis=1)

##REMOVING ALL NULL ROWS
bad_index = df[df['brand'].isnull()].index
df.drop(bad_index, inplace = True)

bad_index = df[df['odo'].isnull()].index
df.drop(bad_index, inplace = True)

bad_index = df[df['cyl'].isnull()].index
df.drop(bad_index, inplace = True)

bad_index = df[df['price'].isnull()].index
df.drop(bad_index, inplace = True)

df.isna().sum()


df['body_type'] = df['body_type'].fillna(value=df['body_type'].mode()[0])
df['cond'] = df['cond'].fillna(value=df['cond'].mode()[0])

In [30]:
##FEATURE ANALYSIS

##REMOVING OUTLIERS FROM YEAR

# Outlier detection 

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

# detect outliers from Price, year & odo
Outliers_to_drop = detect_outliers(df,0,["price","year","odo"])

df.loc[Outliers_to_drop] # Show the outliers rows
df = df.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)

In [31]:
##ONE-HOT ENCODING
df = pd.get_dummies(df, columns=['brand','fuel','cond','title','trans','body_type'])
df.head()

data_train, data_test = train_test_split(df, test_size = 0.3, random_state = 12)

y = df.loc[:,['price']]
x = df.drop(['price'],axis=1)
x = x.drop(['_id','url','full_car_brand','img_url'],axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 12)


In [ ]:
##XGBOOST

n_iter_search = 50


param_grid = {
    'n_estimators': [500, 1000, 2500],
    'eta': [0.1, 0.25, 0.75, 1],
    'reg_alpha':[0.75, 0.5, 0.3 ,0.2, 0.1, 1], 
    'booster': ['gbtree'],
    'reg_lambda': [1e-5, 1e-2, 0.1, 1],
    'scale_pos_weight':[ 5, 10, 25],
}


model = XGBRegressor()
grid_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter = n_iter_search, cv=5, scoring ='neg_mean_squared_error')
grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print(mse)
r2 = r2_score(y_test, y_pred)
print(r2)


In [ ]:
#####################LASSO REGRESSION################################################################
param_grid2 = {'alpha': (np.logspace(-5, 5, 100))} 

model2 = Lasso(normalize=True)
grid_search2 = RandomizedSearchCV(estimator=model, param_distributions=param_grid2, n_iter=n_iter_search, cv=5, scoring='neg_mean_squared_error')
grid_search2.fit(x_train, y_train)
best_params2 = grid_search2.best_params_
best_model2 = grid_search2.best_estimator_
y_pred2 = best_model2.predict(x_test)
mse2 = mean_squared_error(y_test, y_pred2)
mse2**0.5

In [ ]:
###################RIDGE REGRESSION###################################################################

model3 = Ridge(normalize=True)
grid_search3 = RandomizedSearchCV(estimator=model, param_distributions=param_grid2, n_iter=n_iter_search, cv=5, scoring='neg_mean_squared_error')
grid_search3.fit(x_train, y_train)
best_params3 = grid_search3.best_params_
best_model3 = grid_search3.best_estimator_
y_pred3 = best_model3.predict(x_test)
mse3 = mean_squared_error(y_test, y_pred3)
mse3**0.5

In [ ]:
###########RANDOM FOREST##############################################################################

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train, y_train)

#Evaluating the model

best_params_rf = rf_random.best_params_
best_model_rf = rf_random.best_estimator_
y_pred_rf = best_model_rf.predict(x_test)
mse4 = mean_squared_error(y_test, y_pred_rf)
print(mse4)
r2 = r2_score(y_test, y_pred2)
print(r2)

rmse = np.sqrt(mse4)
print(rmse)

print(best_params_rf)
